+ 数据源 [腾讯疫情实时追踪](https://news.qq.com/zt2020/page/feiyan.htm?from=timeline&isappinstalled=0)

+  导入相关模块

In [1]:
import time 
import json
import requests
from datetime import datetime
import pandas as pd 
import numpy as np

+ 抓取每个城市的相关人数

In [4]:
def catch_cityinfo():
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=wuwei_ww_area_counts&callback=&_=%d'%int(time.time()*1000)
    city_data = requests.get(url=url).json()['data']
    city_data = json.loads(city_data)
    city_data = pd.DataFrame(city_data)
    return city_data

In [5]:
citydata = catch_cityinfo()
citydata.head()

,area,city,confirm,country,dead,heal,suspect
0,湖北,武汉,2261,中国,129,51,0
1,湖北,黄冈,496,中国,12,2,0
2,湖北,孝感,399,中国,6,0,0
3,湖北,荆门,191,中国,4,0,0
4,湖北,恩施州,66,中国,0,0,0


+ 绘制国内疫情地图

In [34]:
area_data = citydata.loc[citydata["country"] == "中国"].groupby("area")["confirm"].sum().reset_index()
area_data.columns = ["area","confirm"]
area_data.head()

,area,confirm
0,上海,102
1,云南,55
2,内蒙古,16
3,北京,111
4,台湾,8


In [35]:
from pyecharts import options as opts
from pyecharts.charts import *
from pyecharts.globals import ThemeType

In [36]:
area_map = Map()
area_map .add("",[list(z) for z in zip(list(area_data["area"]), list(area_data["confirm"]))], "china",is_map_symbol_show=False)
area_map.set_global_opts(title_opts=opts.TitleOpts(title="Map-疫情地图"),visualmap_opts=opts.VisualMapOpts(max_=3000,is_piecewise=True,
                pieces=[
                 {"min": "0", "max": "10", "label": "0-10", "color": "#FFC0CB"},
                 {"min": "10", "max": "100", "label": "10-100", "color": "#F08080"},
                 {"min": "100", "max": "1000", "label": "100-1000", "color": "#FF0000"},
                 {"min": "1000", "max": "10000", "label": ">1000", "color": "#8b0000"}
             ]))
area_map.render_notebook()

+ 绘制世界疫情地图

In [37]:
world_data = citydata.groupby("country")["confirm"].sum().reset_index()
world_data.columns = ["country","confirm"]
# 这里有坑，中文地图不显示，要把国家名字替换为英文
world_name = pd.read_excel("世界各国中英文对照.xlsx")
world_data_ = pd.merge(world_data,world_name,left_on ="country",right_on = "中文",how="inner")
world_data_.head()

,country,confirm,英文,中文
0,中国,7488,China,中国
1,加拿大,2,Canada,加拿大
2,尼泊尔,1,Nepal,尼泊尔
3,德国,4,Germany,德国
4,斯里兰卡,1,Sri Lanka,斯里兰卡


In [38]:
world_map = Map()
world_map.add("",[list(z) for z in zip(list(world_data_["英文"]), list(world_data_["confirm"]))], "world",is_map_symbol_show=False)
world_map.set_global_opts(title_opts=opts.TitleOpts(title="2019_nCoV-世界疫情地图"),visualmap_opts=opts.VisualMapOpts(max_=5),legend_opts=opts.LegendOpts(is_show=False))
world_map.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
world_map.render_notebook()

+ 抓取每日确诊和死亡数据

In [39]:
def catch_daily():
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=wuwei_ww_cn_day_counts&callback=&_=%d'%int(time.time()*1000)
    data = json.loads(requests.get(url=url).json()['data'])
    data.sort(key=lambda x:x['date'])
    return data

In [41]:
daily_data = pd.DataFrame(catch_daily())
daily_data.to_excel("daily_data.xlsx")
daily_data.head()

,confirm,date,dead,heal,suspect
0,41,01.13,1,0,0
1,41,01.14,1,0,0
2,41,01.15,2,5,0
3,45,01.16,2,8,0
4,62,01.17,2,12,0


In [16]:
line = Line()
line.add_xaxis(list(daily_data["date"]))
line.add_yaxis("heal",list(daily_data["heal"]))
line.add_yaxis("dead", list(daily_data["dead"]))
line.set_global_opts(title_opts=opts.TitleOpts(title="Line-治愈与死亡趋势"))
line.render_notebook()

In [29]:
def bar() -> Bar:
    c = (
        Bar({"theme": ThemeType.MACARONS})
        .add_xaxis(list(daily_data["date"]))
        .add_yaxis("confirm", list(daily_data["confirm"]))
        .add_yaxis("suspect", list(daily_data["suspect"]))
        .set_global_opts(
            title_opts={"text": "Bar-患病人数增长趋势"}
        )
    )
    return c
bar().render_notebook()